In [241]:
#! usr/bin/env python3
from tqdm import tqdm_notebook as tqdm
import praw
import pandas as pd
import datetime as dt
from PIL import Image
import urllib.request
import wikipedia
import feather
import random
import shutil

import requests
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen

import re
import glob
import time

import ssl

ssl._create_default_https_context = ssl._create_unverified_context

import IPython.display as ipd
import pyttsx3
import os
import uuid
import operator

from shutil import copyfile

from pydub import AudioSegment
from google.cloud import texttospeech

Image.MAX_IMAGE_PIXELS = None

In [242]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="/Users/anthonycelio/xx/My_Projectectect-ef53cc7c18c4.json"

In [ ]:
reddit = praw.Reddit(client_id='ZWkdQHV89SHfEw', \
                     client_secret='xxxxx', \
                     user_agent='Chester', \
                     username='xxxxxx', \
                     password='xxxxxx')

subreddit = reddit.subreddit('Unexpected')
#subreddit = reddit.subreddit('cringeanarchy')

#top_subreddit = subreddit.hot()
top_subreddit = subreddit.top('week')

#top_subreddit = subreddit.top(limit=50)
#for submission in subreddit.hot(limit=1):
#    print(submission.title, submission.id)
    
topics_dict = { "title":[],
               "score":[],
               "id":[],
               "url":[],
               "comms_num":[],
               "created": [],
               "body":[],
               "vid":[]
                  }

for submission in top_subreddit:
    topics_dict["title"].append(submission.title)
    topics_dict["score"].append(submission.score)
    topics_dict["id"].append(submission.id)
    topics_dict["url"].append(submission.url)
    topics_dict["comms_num"].append(submission.num_comments)
    topics_dict["created"].append(submission.created)
    topics_dict["body"].append(submission.selftext)
    topics_dict["vid"].append(submission.media)
    
topics_data = pd.DataFrame(topics_dict)

In [ ]:
#submission.media['reddit_video']['fallback_url']

In [ ]:
topics_data

In [ ]:
topics_data['comms_num'][5]

In [ ]:
brk = 0
use = []
for i in range(100):
    if topics_data["comms_num"][i] >= 13:
        if topics_data["vid"][i] != None:
            try:
                xex = topics_data["vid"][i]["reddit_video"]["duration"]
                if xex > 4 and xex < 30:
                    use.append(i)
                    brk += 1
            except:
                pass
    
    #if brk >= 5:
     #   break

In [ ]:
len(use)

In [ ]:
z99 = 0
for z, i in enumerate(topics_data['url']):
    if z99 < 10:
        if i[-3:] == 'jpg':
            urllib.request.urlretrieve(i, i[-10:])
            z99 += 1
            print(z, i)

In [ ]:
topics_data.loc[0]

In [ ]:
!pwd

In [ ]:
def ret_com(num):
    po = topics_data['id'][num]
    submission = reddit.submission(id=po)
    submission.comments.replace_more(limit=1)
    chop = []
    
    for top_level_comment in submission.comments:
        chop.append([top_level_comment.body])
        for i in (top_level_comment.replies):
            chop.append([i.body])
            for x in (i.replies):
                chop.append([x.body])
                for z in (x.replies):
                    chop.append([z.body])
                    for cc in (z.replies):
                        chop.append([cc.body])
                        for wo in (cc.replies):
                            chop.append([wo.body])
                            for zoo in (wo.replies):
                                chop.append([zoo.body])

    grab = []
    for i in chop:
        if len(i[0]) > 10:
            if len(i[0]) < 300:
                for x in i:
                    i = re.sub(r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*', '', x)
                    i = re.sub(r'[^a-zA-Z1-9.,()\'"?! ]+', '', i)
                    i = i.replace('ITAP', 'I took a picture')

                    grab.append(i)    
    #[topics_data['title'][num]] + grab
    frst = topics_data['title'][num].replace('ITAP of ', '')
    grab.insert(0, frst.replace('ITAP ', ''))

    return grab[:24]

In [ ]:
cc = ret_com(1)

In [ ]:
cc

In [ ]:
len(cc)

In [ ]:
if cc[-1:][0][-1:] == '?':
    cc = cc[:-1]
if cc[-1:][0][-2:] == '?':
    cc = cc[:-1]

In [ ]:
cc[:-1]

In [ ]:
def TTS_lena(str, wave_name):
    client = texttospeech.TextToSpeechClient()
    synthesis_input = texttospeech.types.SynthesisInput(text=str)

    voice = texttospeech.types.VoiceSelectionParams(
        language_code='en-AU',
        name='en-AU-Wavenet-A',
        ssml_gender=texttospeech.enums.SsmlVoiceGender.FEMALE)

    audio_config = texttospeech.types.AudioConfig(
        speaking_rate=0.91,
        pitch=0,
        audio_encoding=texttospeech.enums.AudioEncoding.MP3)
    response = client.synthesize_speech(synthesis_input, voice, audio_config)
    with open('output.mp3', 'wb') as out:
        out.write(response.audio_content)
    newAudio = AudioSegment.from_mp3('output.mp3')
    newAudio.export(wave_name + '.wav', format="wav")
    os.remove('output.mp3')

In [243]:
def TTS_stac(str, wave_name):
    client = texttospeech.TextToSpeechClient()
    synthesis_input = texttospeech.types.SynthesisInput(text=str)

    voice = texttospeech.types.VoiceSelectionParams(
        language_code='en-US',
        name='en-US-Wavenet-F',
        ssml_gender=texttospeech.enums.SsmlVoiceGender.FEMALE)

    audio_config = texttospeech.types.AudioConfig(
        speaking_rate=0.89,
        pitch=-1,
        audio_encoding=texttospeech.enums.AudioEncoding.MP3)
    response = client.synthesize_speech(synthesis_input, voice, audio_config)
    with open('output.mp3', 'wb') as out:
        out.write(response.audio_content)
    newAudio = AudioSegment.from_mp3('output.mp3')
    newAudio.export(wave_name + '.wav', format="wav")
    os.remove('output.mp3')

In [ ]:
def wav_2_phone(wav_file):
    new = AudioSegment.from_wav(wav_file)
    new = new.apply_gain(12).low_pass_filter(30000).high_pass_filter(30000)
    new = new + 15
    new.export(wav_file, format="wav")

In [ ]:
wdir = '/Users/anthonycelio/Documents/deep_news/!machine/corespondants/16/'
if not os.path.exists(wdir): os.makedirs(wdir)

In [ ]:
os.chdir('/Users/anthonycelio/Documents/deep_news/correspondants/lena/general_bumpers/')

In [ ]:
TTS_stac('''Thank you for joining us Lena. We'll talk to you again soon.''', 'St_out')

In [ ]:
(use[10:14])

In [ ]:
for m in tqdm(range(len(use[10:15]))):
    t = str(int(time.time()))
    sha = wdir+t+'/'
    os.makedirs(sha)
    os.chdir(sha)

    cc = ret_com(use[10:15][m])[:20]
    if cc[-1:][0][-1:] == '?':
        cc = cc[:-1]
    if cc[-1:][0][-2:] == '?':
        cc = cc[:-1]
    mp4s = topics_data["vid"][use[10:15][m]]["reddit_video"]["fallback_url"]
    urllib.request.urlretrieve(mp4s, mp4s[-14:]+'.mp4')
    for z, i in enumerate(tqdm(cc)):
        if z % 2 == 0:
            TTS_lena(i, (str(z)).zfill(2))
            wav_2_phone((str(z)).zfill(2)+'.wav')
        else:
            TTS_stac(i, (str(z)).zfill(2))

In [ ]:
su = '''The Parasol (also known as El Quitasol) is one of a cartoon series of oil on linen paintings made by the painter Francisco Goya. This series of paintings was specifically made in order to be transformed into tapestries that would be hung on the walls of the Royal Palace of El Pardo in Madrid, Spain. The tapestries showed serene events in everyday life, which made them a nice addition to the dining room of Prince and Princess of Asturias—the future King Charles IV and Maria Luisa of Parma. The queen called on Goya because she wanted to decorate the dining room with cheerful scenes; The Parasol and the other tapestry paintings were Goya's response to this request. The painting is currently located in the Museo del Prado in Madrid as is another in the series, Blind man's bluff. '''

In [ ]:
TTS_stac(su, '/Users/anthonycelio/Desktop/goya')

In [ ]:
sea = [int(i) for i in os.listdir(wdir) if len(i) is 10]

In [ ]:
sea

In [ ]:
print('''var vs = '''+'"'+str(wdir)+'"'+'''
var myblocnFolder = new Folder(vs).getFiles()
lena = '''+str(sea))

In [ ]:
wav_2_phone((str(z)).zfill(2)+'.wav')

In [ ]:
lena = '/Users/anthonycelio/Documents/deep_news/correspondants/lena/'

In [ ]:
zep = wikipedia.page(imago)

In [ ]:
inter = random.randint(0, 430)

In [ ]:
imago = str(mydivs[inter].findAll('a')[0]).rsplit(' ')[1][12:-1]
imagog = str(mydivs[inter].findAll('a')[0]).rsplit(' ')[0][12:-1]
if len(imago) < len(imagog):
    imago = imagog
blerb(wikipedia.page(imago))

In [ ]:
zzz = wikipedia.page(imago).content

In [ ]:
len(os.listdir(wdir))

In [ ]:
murm = os.listdir(wdir)[random.randint(0, 334)]

In [ ]:
wdir = '/Users/anthonycelio/Documents/deep_news/!machine/art_set/Wiki2/'

In [ ]:
buhz = [] 
for iu in tqdm(os.listdir(wdir)):
    try:
        zxx = '/Users/anthonycelio/Documents/deep_news/!machine/art_set/'
        path = zxx+'Wiki2/'+iu+'/my_data.feather'
        wdf = feather.read_dataframe(path)
        zep = wikipedia.page(wdf.loc[0][0])
        wdf.at(0)[4] = blerb(zep) 
        feather.write_dataframe(wdf, path)
    except:
        buhz.append(iu)
        pass

In [ ]:
wdf

In [ ]:
wdf.loc[5][0]

In [ ]:
wdf

In [ ]:
wdf.at(0)[2] = blerb(zep)

In [ ]:
wdf.loc[0][2][0]

In [ ]:
[i for i in (zep.content.split("==")[0], zep.content.split("==")[2])]

In [ ]:
muh = 'https://en.wikipedia.org/wiki/Wikipedia:Featured_pictures/Artwork/Paintings'

In [ ]:
req = Request(muh, headers={'User-Agent': 'Mozilla/5.0'})
webpage = urlopen(req).read()
soup = BeautifulSoup(webpage, "html.parser")

In [ ]:
mydivs = soup.findAll("div", {"class": "mw-parser-output"})

In [ ]:
mydivs[0].find('p').text

In [ ]:
def blerb(intext):
    zo = intext.content
    zo = re.sub(r'==.+==', '', zo)
    zo = re.sub(r'(?<!\d)\d{4}(?!\d)', '', zo)

    zo = zo.replace("....", ". ")
    zo = zo.replace("\"", ". ")
    zo = zo.replace("\n", ". ")

    zo = zo.replace(".......", ".")
    zo = zo.replace("......", ".")
    zo = zo.replace(".....", ".")
    zo = zo.replace("....", ".")
    zo = zo.replace("...", ".")
    zo = zo.replace("..", ".")

    zo = zo.replace(". . . . . . . ", ". ")
    zo = zo.replace(". . . . . . ", ". ")
    zo = zo.replace(". . . . . ", ". ")
    zo = zo.replace(". . . . ", ". ")
    zo = zo.replace(". . . ", ". ")
    zo = zo.replace(". . . ", ". ")
    zo = zo.replace(". . ", ". ")
    zo = zo.replace("..", ". ")

    und = 0
    bruh = ""
    if len(zo) > 400:
        for i in zo.rsplit('.'):
            if und < 2500:
                bruh += i+'. '
                und += len(i)
        bruh += '.'
    zo = bruh.replace(". . ", ". ")
    zo = zo.replace(".  ", ". ")
    zo = zo.replace(". .", ".")
    
    return zo

In [ ]:
wdir = '/Users/anthonycelio/Documents/deep_news/!machine/art_set/Wiki2/'

In [ ]:
missed2 = []
for ii in tqdm(range(494)):
    try:
        imago = str(mydivs[ii].findAll('a')[0]).rsplit(' ')[1][12:-1]
        imagog = str(mydivs[ii].findAll('a')[0]).rsplit(' ')[0][12:-1]
        if len(imago) < len(imagog):
            imago = imagog

        zep = wikipedia.page(imago)
        x3x = blerb(zep)
        if len(x3x) > 400: 
            wdf = pd.DataFrame([imago, x3x])

            t = str(uuid.uuid4())[-12:]
            sha = wdir+t+'/'
            os.makedirs(sha)
            os.chdir(sha)

            path = sha+'my_data.feather' 
            feather.write_dataframe(wdf, path)

            for m in zep.images:
                urllib.request.urlretrieve(m, sha+str(uuid.uuid4())[-5:]+m[-14:])
            d = {}
            for i in os.listdir(sha):
                cu = os.stat(i).st_size
                d[i] = cu
            for x in os.listdir(sha):
                if x != max(d.items(), key=operator.itemgetter(1))[0]:
                    if x[-4:] != 'ther':
                        os.remove(x)

            emage = max(d.items(), key=operator.itemgetter(1))[0]
            img = Image.open(emage)
            sz = os.stat(emage).st_size
            if sz > 9e+7:
                new_img = img.resize((int((9e+7/sz) * img.width), int((9e+7/sz) * img.height)))
                new_img.save(emage, "JPEG", optimize=False)

    except:
        missed2.append(imago)
        pass

In [ ]:
wdf = feather.read_dataframe('/Users/anthonycelio/Documents/deep_news/!machine/art_set/Wiki2/acacf1ab7352/my_data.feather')

In [ ]:
wdf.loc[1][0]

In [ ]:
path = '/Users/anthonycelio/Documents/deep_news/!machine/art_set/Wiki2/d5e3ede6c7f0/my_data.feather'

In [ ]:
wdf = feather.read_dataframe(path)

zep = wikipedia.page(wdf.loc[0][0])

#wdf.loc[2][0] = 

In [ ]:
wdf

In [ ]:
wdf.loc[1][0]

In [ ]:
zep.content

In [ ]:
[i for i in (zep.content.split("==")[0], zep.content.split("==")[2])]

In [ ]:
os.chdir('/Users/anthonycelio/Documents/deep_news/!machine/art_set/Wiki2/d5e3ede6c7f0/')

In [ ]:
wdf = feather.read_dataframe('my_data.feather')

In [ ]:
TTS_stac(wdf.loc[2][0], 'TTS')

In [ ]:
time.sleep(0.5)

In [ ]:
len(missed) * 0.5

In [ ]:
sha = '/Users/anthonycelio/Documents/deep_news/!machine/art_set/9ada7545c580/'

In [ ]:
os.chdir(sha)

In [ ]:
d = {}
for i in os.listdir(sha):
    cu = os.stat(i).st_size
    d[i] = cu

In [ ]:
d

In [ ]:
for x in os.listdir(sha):
    if x != max(d.items(), key=operator.itemgetter(1))[0]:
        os.remove(x)

In [ ]:
len(os.listdir(wdir))

In [ ]:
missed

In [ ]:
imago

In [ ]:
imagog

In [ ]:
zep = wikipedia.page(imago)

In [ ]:
len('direct')

In [ ]:
wdf = pd.DataFrame([blerb(zep)])

In [ ]:
wdf

In [ ]:
path = wdir+'my_data.feather' 
feather.write_dataframe(wdf, path)
wdf = feather.read_dataframe(path)

In [ ]:
fea = '/Users/anthonycelio/Documents/deep_news/!machine/art_set/401c8e3bcb98/my_data.feather'

In [ ]:
furv = feather.read_dataframe(fea)

In [ ]:
furv.loc[0][0]

In [ ]:
shutil.rmtree

In [ ]:
len(os.listdir(wdir))

In [ ]:
for i in os.listdir(wdir):
    try:
        bub = wdir+i+'/'
        if len(os.listdir(bub)) != 2:
            shutil.rmtree(bub)
    except:
        print('I had to pass')
        pass

In [ ]:
emage = '/Users/anthonycelio/Documents/deep_news/NewsFiles/2/nian.jpg'

In [ ]:
# W x H
new_img = img.resize((int(0.3 * img.width), int(0.3 * img.height)))
new_img.save(werdir+"resized.jpg", "JPEG", optimize=False)

In [ ]:
len(os.listdir(wdir))

In [244]:
wdir = '/Users/anthonycelio/Documents/deep_news/!machine/art_set/Wiki4/'

In [245]:
os.chdir(wdir)

In [246]:
tried = []
for i in tqdm(os.listdir(wdir)[1:]):
  #  try:
    os.chdir(wdir+i)
    wdf = feather.read_dataframe(wdir+i+'/my_data.feather')
    TTS_stac(wdf.loc[1][0], 'TTS')
#     except:
#         tried.append(i)
#         pass

In [8]:
tried

[]

In [ ]:
TTS_stac()

In [259]:
wedir = '/Users/anthonycelio/Documents/deep_news/NewsFiles/!WikiArt/'

In [260]:
#CREATE NEW FILES BETWEEN 000 AND 999 WITH A DIR NAMED '1' AND '2' IN EACH
for i in range(1000):
    os.makedirs(wedir+str(i).zfill(3)+'/2/')
    os.makedirs(wedir+str(i).zfill(3)+'/1/')

In [ ]:
fromfile = '/Users/anthonycelio/Documents/deep_news/NewsFiles/'

In [ ]:
fromlist = glob.glob(fromfile+'*.jpg')

In [ ]:
#Move random images into poem folders with new name
i = 0
chum = 14
while i < len(fromlist):
    os.rename(fromlist[i], wedir+str(chum).zfill(3)+'/2/'+str(i)+'.jpg')
    i += 1
    os.rename(fromlist[i], wedir+str(chum).zfill(3)+'/1/'+str(i)+'.jpg')
    i += 1
    chum += 1

In [ ]:
#Copy art tts and jpg into AE ready folders
i = 0
chum = 14
while i < len(fromlist):
    shutil.copyfile(fromlist[i], wedir+str(chum).zfill(3)+'/2/'+str(i)+'.jpg')
    i += 1
    shutil.copyfile(fromlist[i], wedir+str(chum).zfill(3)+'/1/'+str(i)+'.jpg')
    i += 1
    chum += 1

In [261]:
fromlist = '/Users/anthonycelio/Documents/deep_news/!machine/art_set/Wiki4/'

In [262]:
dest = '/Users/anthonycelio/Documents/deep_news/NewsFiles/!WikiArt/'

In [263]:
# PUT ART PHOTO AND WAVE IN 1 AND 2 FILE FOLDERS. BEGIN AT 000
f = 1
chum = 0
for z in tqdm(glob.glob(fromlist+'/*')):
    for zc in glob.glob(z+'/*.jpg'):
        shutil.copyfile(zc, dest+str(chum).zfill(3)+'/'+str(f)+'/'+zc[-10:])
    for zc in glob.glob(z+'/*.wav'):
        shutil.copyfile(zc, dest+str(chum).zfill(3)+'/'+str(f)+'/'+zc[-7:])
    for zc in glob.glob(z+'/*.feather'):
        shutil.copyfile(zc, dest+str(chum).zfill(3)+'/'+str(f)+'/'+zc[-11:])
    f += 1
    if f == 3:
        f = 1
        chum += 1

In [264]:
396*0.5

198.0

In [47]:
'asdfasdfasdfasdf123412'[-3:]

'412'

In [91]:
wdir = '/Users/anthonycelio/Documents/deep_news/!machine/art_set/Wiki4/'

In [94]:
td = os.listdir(wdir)
wdf = feather.read_dataframe(wdir+'17bcf7c3ae83'+'/my_data.feather')
wdf.loc[0][0]

'The_Dog_(Goya)'

In [ ]:
wdf.loc[0][0]

In [80]:
w3 = '/Users/anthonycelio/Documents/deep_news/!machine/art_set/Wiki3/'
w4 = '/Users/anthonycelio/Documents/deep_news/!machine/art_set/Wiki4/'

In [84]:
w3g = glob.glob(w3+'/*/')
w4g = glob.glob(w4+'/*/')

In [90]:
#CHECK PIECE TITLE AND COPY WAV FROM OTHER FILE OF SAME TITLE
for i in tqdm(w3g):
    w3df = feather.read_dataframe(i+'/my_data.feather')
    for z in w4g:
        w4df = feather.read_dataframe(z+'/my_data.feather')
        if w3df.loc[0][0] == w4df.loc[0][0]:
            shutil.copyfile(i+'TTS.wav', z+'TTS.wav')

In [89]:
wdf3.loc[0][0]

'The_Storm_on_the_Sea_of_Galilee'

In [96]:
os.chdir(w4)

In [214]:
wwww = '/Users/anthonycelio/Documents/deep_news/!machine/art_set/Wiki4/'

In [215]:
w4g = glob.glob(wwww+'*')

In [185]:
vov = glob.glob(wwww+'*')

In [196]:
len('00e14dfe1d4c')

12

In [ ]:
w4g

In [217]:
#IF A DIR CONTAINS A FILE OF X FORMAT MOVE THE DIR
for zxz in w4g:
    br = os.listdir(zxz)
    for zz in br:
        if zz.endswith('wav'):
            os.rename(zxz, '/Users/anthonycelio/Documents/deep_news/!machine/art_set/Wiki4_filter/'+zxz[-12:]+'/')
            pass

In [ ]:
for i in w4g:
    if not os.path.exists(glob.glob(i+'/*.jpg')[0]):
        os.remove(i)

In [231]:
wwww = '/Users/anthonycelio/Documents/deep_news/!machine/art_set/Wiki4_filter/'
w4g = glob.glob(wwww+'*')

In [230]:
w4g

[]

In [240]:
for zxz in w4g:
    br = os.listdir(zxz)
    if len(br) < 2:
        print ('why')
        print(zxz)

In [228]:
if len(br) != 3:
    print ('why')

In [ ]:
wdir+str(os.listdir(wdir)[9])

In [ ]:
wdir = '/Users/anthonycelio/Documents/deep_news/!machine/art_set/Wiki3/'

In [ ]:
saw = 0
for i in tqdm(os.listdir(wdir)[1:]):
    try:
        os.chdir(wdir+str(i))
        newAudio = AudioSegment.from_mp3('output.mp3')
        newAudio.export('TTS.wav', format="wav")
        os.remove('output.mp3')
    except:
        saw += 1
        pass

In [ ]:
saw

In [ ]:
wdir

In [ ]:
len(os.listdir(wdir))

In [ ]:
'/Users/anthonycelio/Documents/deep_news/!machine/art_set/Wiki3/000d6564edb1/TTS.wav'[-21:-7]

In [9]:
saw = 0
for i in tqdm(os.listdir(wdir)):
    try:
        xox = wdir+str(i)+'/TTS.wav'
        if os.path.exists(xox):
            os.rename(xox[:-7], '/Users/anthonycelio/Documents/deep_news/!machine/art_set/Wiki3_Ready'+xox[-21:-7])
    except:
        saw += 1
        pass

In [10]:
saw

0